In [1]:
import pathlib
import urllib
import sys
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, iqr, skew, kurtosis

# 1. Collect data

In [6]:
path = pathlib.Path('../data/adult.csv')
if path.exists():
    print('adult dataset found!')
else:
    sys.stdout.write('Downloading the adult dataset from the Internet...')
    ADULTURL = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    urllib.request.urlretrieve(ADULTURL, path.absolute())
    sys.stdout.write('Done!')

adult dataset found!


# 2. Understand the data in context


In [7]:
# Load the adult dataset into a Pandas dataframe
adult_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', '>50K']
adult = pd.read_csv(path.absolute(),names=adult_columns)
for i in set(adult['education']):
    print(i)

 10th
 7th-8th
 Prof-school
 9th
 5th-6th
 HS-grad
 Some-college
 Assoc-acdm
 Bachelors
 1st-4th
 12th
 Doctorate
 Assoc-voc
 11th
 Preschool
 Masters


# 3. Understand the data
### 3.1 Describe the meansing and type for each attribute
 - **age**: numeric, continuous. 
 - **workclass**: categorical, discrete.
 - **fnlwgt**: numeric, continuous
 - **education**: ordinal, discrete
 - **education-num**: numeric, continuous
 - **marital-status**: categorical, discrete
 - **occupation**: categorical, discrete
 - **relationship**: categorical, discrete
 - **race**: categorical, discrete
 - **sex**: categorical, binary
 - **capital-gain**: numeric, continuous
 - **capital-loss**: numeric, continuous
 - **hours-per-week**: numeric, continuous
 - **native-country**: categorical, discrete
 - **>50k**: categorical, binary

*3.1 subsection: explanation for non-self-explanatory attributes*
 - **fnlwgt**: represents final weight. according to [2], it's the number of units that this record could represent in the 
 target population. In [1], the author explained that the final weight is controlled by three factors: a single cell estimatation
 of the population 16+ for each state; controls for Hispanic Origin by age and sex; controls by race, age and sex.
 - **education_num**: represents the number of years of education in total.[2]
 - **relationship**: represents the individual's role in it's family.[2]
 - **capital_gain** and **capital_loss**: represents the income and loss from non-salary ways, e.g. investment.[2]
 - **education**: due to lack of domain knowledge in U.S. education system, we decided to refer to [3].
 In which they order the education level in the following manner:  
 Preschool < 1st-4th < 5th-6th < 7th-8th < 9th < 10th < 11th < 12th < HS-grad < Prof-school < Assoc-acdm < Assoc-voc < Some-college < Bachelors < Masters < Doctorate.  
 [3] further merges  "Preschool", "1st-4th", "5th-6th", "7th-8th", "9th", "10th", "11th" and "12th" groups to dropout group,
 "Assoc-acdm" and "Assoc-voc" groups to "Associates" group,“HS-grad” and “Some-college” groups to “HS-Graduate” group.  
 For us, we would adopt this strategy, merging Prof-school into HS-Graduate group, and instead of arranging the education
 level like shown above, we have the following order:  
 dropout < hs-graduate < associates < bachelor < master < doctorate.
 
### 3.2 Verify data quality
#### 3.2.1 duplication

In [8]:
# Check duplication and drop duplication via pandas built in function
check_duplication = adult.duplicated()
duplicated = adult[check_duplication]
print("There are "+str(len(duplicated.index))+" duplicated data entries found.(One copy of the data entry will be kept in the dataset)")
adult = adult.drop_duplicates()
print("After drop duplicate, there are "+str(len(adult.index))+ " data entries remains.")
adult = adult.reset_index(drop=True)

There are 24 duplicated data entries found.(One copy of the data entry will be kept in the dataset)
After drop duplicate, there are 32537 data entries remains.


#### 3.2.2 missing values

In [9]:
# drop the indexes for which column has value '?'
unknown_row =[]

for i in range(len(adult.index)):
    if ('?' in adult.iloc[i]['workclass']) or ('?' in adult.iloc[i]['occupation']) or ('?' in adult.iloc[i]['native-country']):
        unknown_row.append(i)
adult = adult.drop(index = unknown_row)
adult = adult.reset_index(drop=True)

In [ ]:
print(set(adult['education']))
# print(adult.loc[adult['education']==' Preschool'])
# print(set(adult['education']))

#### 3.2.3 finding outliers

To find outlier, we only focus on numeric values
As we can see in the following, the calculated lower bound and upper bound for both capital_gain and capital loss is both
0. In 3.3 section we will see that the median for these two value are 0 either. Which means in this dataset most of the people
doesn't do investment and thus investment does not incur gain or loss to their income. So for this two attribute we just
don't do anything to the outliers

In [11]:
# we calculate interquantile range add use it as our guide line as detecting outlier

adult_numrics = adult.select_dtypes(include=['int64'])
Q1 = adult_numrics.quantile(0.25)
Q3 = adult_numrics.quantile(0.75)
iqr = Q3-Q1
# print(iqr)
# print(adult_numrics['capital-gain'])
lower_bound = Q1 - 1.5 * iqr
upper_bound = Q3 + 1.5 * iqr
print("By calculating interquantile range, we can get the expected lower/upper bound for each numeric value")
adult_numrics = adult_numrics.drop(['capital-gain','capital-loss'],axis='columns')
lower_bound.drop(['capital-gain','capital-loss'],inplace=True)
upper_bound.drop(['capital-gain','capital-loss'],inplace=True)
print("As explained before we drop the catipal-gain and capital-loss condition, and thus we have")
outliers = adult_numrics[((adult_numrics<lower_bound) | (adult_numrics>upper_bound)).any(axis=1)]
print("lower bound as:")
print(lower_bound)
print("and upper bound as:")
print(upper_bound)
print("After reviewing this calculated lower and upper bound, we decided to change some of the value. Since the result")
print("value would be an outlier according to the interquantile range, but still a reasonable value in reality")
print("The lower bound of hours-per-week is set to 20, which is the expected value for a part-time job")
print("The upper bound of age is set to 85, which is longer than the life expectancy in U.S., and around the value of ")
print("life expectancy in Canada")
print("The upper bound of hours-per-week is set to 72, which is calculated according to the \'famous\' 9-9-6 working load")
print("in China.")
print("The upper bound of education-num is set to 26, which is a little bit longer than the expected year of education of")
print("a Ph.D., which is 6 years for elementary, 6 years for middle and high school, 4 year undergraduate, 2 year master")
print("and 5 year Ph.D. studentship, add up to 23 years.")

lower_bound['hours-per-week'] = 20
upper_bound['age']=85
upper_bound['hours-per-week']=72
upper_bound['education-num'] = 26
outliers = adult_numrics[((adult_numrics<lower_bound) | (adult_numrics>upper_bound)).any(axis=1)]
print("After applying such filter, we found %d outliers in the dataset, which is a pretty large portion of the dataset"%len(outliers.index))
adult_inliers = adult.drop(outliers.index)
print("Consider the number of the outliers is quite big, we will just keep them inside the dataset and have a closer")
print("look in section 4 Exceptional Work")

By calculating interquantile range, we can get the expected lower/upper bound for each numeric value
As explained before we drop the catipal-gain and capital-loss condition, and thus we have
lower bound as:
age                  -0.5
fnlwgt           -62338.0
education-num         3.0
hours-per-week       32.5
dtype: float64
and upper bound as:
age                   75.5
fnlwgt            417570.0
education-num         19.0
hours-per-week        52.5
dtype: float64
After reviewing this calculated lower and upper bound, we decided to change some of the value. Since the result
value would be an outlier according to the interquantile range, but still a reasonable value in reality
The lower bound of hours-per-week is set to 20, which is the expected value for a part-time job
The upper bound of age is set to 85, which is longer than the life expectancy in U.S., and around the value of 
life expectancy in Canada
The upper bound of hours-per-week is set to 72, which is calculated according to 

In [12]:
"""
Numericalize the categorical columns
"""
# mapping dictioniary for education row, we take education as a ordinal attribute, as was explained earlier
education_mapping_dict = {' Preschool':1, ' 1st-4th':1, ' 5th-6th':1, ' 7th-8th':1, ' 9th':1, ' 10th':1, ' 11th':1, ' 12th':1,
                          ' HS-grad':2, ' Prof-school':2, ' Some-college':2,
                          ' Assoc-voc':3, ' Assoc-acdm':3,
                          ' Bachelors':4,
                          ' Masters':5,
                          ' Doctorate':6}
categorical_list=['workclass','marital-status','occupation','relationship','race','sex','native-country']
def categorical_is_converted(input_dtype):
    """
    check if the target categorical value is converted to some integer value
    :param input_dtype: the datatype of the input column
    :return: True if the dtype is some kind of integer, False otherwise
    """
    return input_dtype in [np.int, np.int64, np.int32, np.long]
def one_hot_batch(df, attribute_list,prefix_list=None):
    """
    :param attribute_list: list of attributes that waiting to be one hotted 
    :param prefix_list: list of prefix for the attribute after one-hot representation, if not set, just use the attribute name
    :return: False if unexpected input found, otherwise return the result dataframe
    """
    if prefix_list is None:
        prefix_list = attribute_list
    if not len(attribute_list) == len(prefix_list):
        return False
    for i in range(len(attribute_list)):
        if not categorical_is_converted(df[attribute_list[i]].dtype):
            dummies = pd.get_dummies(df[attribute_list[i]], prefix=prefix_list[i])
            df = df.drop(columns=[attribute_list[i]])
            df = pd.concat([df,dummies], axis=1)
    return df
# map education to integers
if not categorical_is_converted(adult['education'].dtype):
    adult = adult.replace({'education': education_mapping_dict})
# convert all non-ordinal categorical attribute using one-hot representation
adult = one_hot_batch(adult, categorical_list)
# map the prediction target: the ultimate income rank into integer. Since it's binary, we don't use one hot representation
if not categorical_is_converted(adult['>50K'].dtype):
    for i in range(len(adult.index)):
        if '>50K' in adult.iloc[i]['>50K']:
            adult.iloc[i, adult.columns.get_loc('>50K')] = 1
        else:
            adult.iloc[i,adult.columns.get_loc('>50K')] = 0
adult.sample(5)

,age,fnlwgt,education,education-num,capital-gain,capital-loss,hours-per-week,>50K,workclass_ Federal-gov,workclass_ Local-gov,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
13817,43,19914,2,10,0,0,15,0,1,0,...,0,0,0,0,0,0,0,1,0,0
7562,18,274057,1,7,0,0,8,0,0,0,...,0,0,0,0,0,0,0,1,0,0
24384,35,160192,2,10,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5390,37,323155,1,2,0,0,85,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17020,18,238867,1,7,0,1602,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
adult.head(5)

### 1. Statistics about Numeric values

In [ ]:
# select all columns with numeric values
adult_numeric = adult.select_dtypes(include=np.number)

In [ ]:
adult_numeric.head(5)

In [ ]:
cnames = ['mean', 'median', 'mode', 'trimmed mean (p=20%)', 'min', 'max', 'range', 'std']
adult_numeric.agg(lambda x: pd.Series([np.mean(x), np.median(x),x.mode()[0], trim_mean(x, 0.2),x.min(),x.max(),x.max()-x.min(),x.std()], index=cnames))

### 2. Statistics about Categorical values

In [ ]:
# select all columns with categorical values
adult_categorical = adult.select_dtypes(include=['object'])

In [ ]:
adult_categorical.head(5)

In [ ]:
cnames = adult_categorical.columns.tolist()
cnames

In [ ]:
adult_categorical['workclass'].value_counts()

In [ ]:
adult_categorical['education'].value_counts()

In [ ]:
adult['marital-status'].value_counts()

In [ ]:
adult_categorical['occupation'].value_counts()

In [ ]:
adult['relationship'].value_counts()

In [ ]:
adult['race'].value_counts()

In [ ]:
adult['sex'].value_counts()

In [ ]:
adult['native-country'].value_counts()

In [ ]:
adult.groupby('workclass').agg(['mean', lambda x: trim_mean(x, 0.2), 'median', 'std']).T

In [ ]:
adult.cov()

# Reference
    [1] Kaggle adult census income dataset. Date accessed: Sept. 2019. url: https://www.kaggle.com/uciml/adult-census-income
    [2] Haojun Zhu, Predicting Earning Potential using the Adult Dataset. Dec. 2016. url: https://rstudio-pubs-static.s3.amazonaws.com/235617_51e06fa6c43b47d1b6daca2523b2f9e4.html
    [3] Bui Dinh Chien, Jean-Daniel Zucker, Complex Data Mining Project. Date accessed: Sept. 2019. url: https://sites.google.com/site/complexdataminingproject/.